**Synopsis - Core:**

For Core use, the create_async_engine() function creates an instance of *AsyncEngine* which then offers an async version of the traditional Engine API. The AsyncEngine delivers an *AsyncConnection* => via its AsyncEngine.connect() and AsyncEngine.begin() methods .The AsyncConnection can then invoke statements AsyncConnection.execute() or AsyncConnection.stream() 

In [1]:
import asyncio

from sqlalchemy import Column, Table, MetaData, select, String, URL , engine
from sqlalchemy.ext.asyncio import create_async_engine

url = URL.create(
    drivername="postgresql+asyncpg", # The asyncio extension requires an async driver to be used. The loaded 'psycopg2' is not async.
    username="postgres",
    host="192.168.0.8",
    port="5432",
    password="1234",
    database="tests"
)
meta = MetaData()
t1 = Table("t1", meta, Column("name", String(50), primary_key=True))

async def async_main() -> None:
    engine = create_async_engine(
        url,
        echo=True,
    )

    print(isinstance(engine , Engine))

    async with engine.begin() as conn:
        await conn.run_sync(meta.create_all)

        await conn.execute(
            t1.insert(), [{"name": "some name 1"}, {"name": "some name 2"}]
        )

    async with engine.connect() as conn:
        # select a Result, which will be delivered with buffered
        # results
        result = await conn.execute(select(t1).where(t1.c.name == "some name 1"))

        print(result.fetchall())

    # for AsyncEngine created in function scope, close and
    # clean-up pooled connections
    await engine.dispose()


asyncio.run(async_main())

RuntimeError: asyncio.run() cannot be called from a running event loop

Above, the *AsyncConnection.run_sync()* method may be used to invoke special DDL functions such as MetaData.create_all() that *don’t include an awaitable* hook.

The AsyncConnection also features a *“streaming”* API via the AsyncConnection.stream() method that returns an AsyncResult object. This result object uses a server-side cursor and provides an async/await API, such as an async iterator:

In [ ]:
async with engine.connect() as conn:
    async_result = await conn.stream(select(t1))

    async for row in async_result:
        print("row: %s" % (row,))